In [11]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [12]:
## Load the trained model, scaler pickle file and onehot encoded file
model=load_model('regression_model.keras')

## load the encoders and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('one_hot_encoder_geography.pkl','rb') as file:
    one_hot_encoder_geograpy=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [13]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'Exited': 0
}

In [14]:
## One Hot encode geography
geo_encoded=one_hot_encoder_geograpy.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=one_hot_encoder_geograpy.get_feature_names_out(['Geography']))
geo_encoded_df

e:\Data science\ANN churn classification project\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [15]:
# converting key value data into dataframe
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,600,France,Male,40,3,60000,2,1,1,0


In [16]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,600,France,1,40,3,60000,2,1,1,0


In [17]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,0,1.0,0.0,0.0


In [25]:
# Required feature columns for prediction
feature_columns = ['CreditScore',
                   'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
                   'HasCrCard', 'IsActiveMember', 'Exited','Geography_France', 'Geography_Germany', 'Geography_Spain']

# Check for missing columns and add them with default values
for col in feature_columns:
    if col not in input_df.columns:
        input_df[col] = 0  # Default value for missing features

In [26]:
# Drop extra columns that are not part of the features
input_df = input_df[feature_columns]

In [27]:
## scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.50857963,  1.00150113,
        -0.57946723, -0.57638802]])

In [28]:
## Predict churn

prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step


array([[91285.87]], dtype=float32)

In [29]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(91285.87)

In [30]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is likely to churn.


In [31]:
'''
@tf.function(input_signature=[tf.TensorSpec(shape=[None, 10], dtype=tf.float32)])
def my_model_fn(input_data):
    return model(input_data)
    '''

'\n@tf.function(input_signature=[tf.TensorSpec(shape=[None, 10], dtype=tf.float32)])\ndef my_model_fn(input_data):\n    return model(input_data)\n    '